In [ ]:
!pip install dalex

     |████████████████████████████████| 962kB 6.2MB/s 
     |████████████████████████████████| 13.2MB 269kB/s 
     |████████████████████████████████| 81kB 5.1MB/s 
  Created wheel for dalex: filename=dalex-1.0.1-cp37-none-any.whl size=978196 sha256=f0aac6259a7d79e63a07e3b6369995c3962d06d15a51519af97585c1bb9c6b08
  Stored in directory: /root/.cache/pip/wheels/50/e9/55/f49b1035158a7d85746a3ba1514f8e31481227087c3dcc222a
Successfully built dalex
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dalex as dx
from copy import deepcopy, copy
from sklearn.model_selection import train_test_split
import warnings
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df  = pd.read_csv('/content/drive/My Drive/Colab Notebooks/heart.csv')

In [ ]:
def encoder_data(df):
  data = df.copy()
  a = pd.get_dummies(data['cp'], prefix = 'cp')
  b = pd.get_dummies(data['thal'], prefix = 'thal')
  c = pd.get_dummies(data['slope'], prefix = 'slope')
  d = pd.get_dummies(data['restecg'], prefix = 'restecg')
  frames = [data.drop(columns = ['cp', 'thal', 'slope', 'restecg']), a, b, c, d]
  data = pd.concat(frames, axis = 1)
  return data

In [ ]:
df.ca.replace(4, 0, inplace=True)
df.thal.replace(0, 2, inplace=True)
df = encoder_data(df)
x = df.drop('target', axis=1)
y = df.target

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2,random_state=1 , stratify = y)

In [ ]:
lr = LogisticRegression()
lr.fit(x_train, y_train)
y_hat = lr.predict(x_test)
print(cross_val_score(lr, x_train, y_train, cv =10, scoring='accuracy'))

[0.84       0.84       0.875      0.875      0.79166667 0.83333333
 0.875      0.83333333 0.91666667 0.91666667]


In [ ]:
explainer = dx.Explainer(lr, x_train, y_train)

Preparation of a new explainer is initiated

  -> data              : 242 rows 22 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 242 values
  -> model_class       : sklearn.linear_model._logistic.LogisticRegression (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7fa0b95474d0> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.000605, mean = 0.544, max = 0.994
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.96, mean = 0.00151, max = 0.969
  -> model_info        : package sklearn

A new explainer has been created!


# Analiza pojedyńczego przypaku
Analizujemy pojedyńczego pacjenta: jest to mężczyzna w wieku 45 lat, ze stosunkowo niskim ciśnieniem krwi za to dużym cholesterolem, jest on faktycznie chory.
Główne zmienne dzięki którym diagnozujemy go jako chorego to:
- Naprawiona wada (thal_2 = 1) warto zwórcić uwagę, że pomimo występowania jako trzecia na wykresie to właśnie ta zminna jest kluczowa ponieważ, (thal 3 = 0) wpływa również na plus diagnozy, a jedno wyklucza występowanie drugiej zmiennej 
- brak naczyń zabarwionych w czsie fluoroskopii (ca)
- występownie bólu w klatce (cp_0 = 0) co ciekawe nie istotne jaki ból się pojwaił ale sam fakt występowania.

Zaskakujące może wydawać się, że duży cholesterol nie wpływa na pozywtyne zdiagnowoanie. Podejrzewam, że wynika to z metody Break Down, wydaje się, że zmienne ciągłe nie są dla niej aż tak istotne.

In [ ]:
x_train.iloc[3]

## 1

In [ ]:
explainer.predict(x_test)[3]

0.929020023984435

## 2

In [ ]:
pp = explainer.predict_parts(x_test.iloc[3])
pp.plot()

In [ ]:
#Kod do znajdowania przypadków do pkt 3
'''
pp1 = explainer.predict_parts(x_test.iloc[11])
ls = []
for i in range(1,x_test.shape[0]):
  pp = explainer.predict_parts(x_test.iloc[i-1])
  if any(pp.result.iloc[:3,0] == 'ca') or any(pp.result.iloc[:3,0] == 'cp_0'):
    continue
  else:
    print(str(i-1))
'''

"\npp1 = explainer.predict_parts(x_test.iloc[11])\nls = []\nfor i in range(1,x_test.shape[0]):\n  pp = explainer.predict_parts(x_test.iloc[i-1])\n  if any(pp.result.iloc[:3,0] == 'ca') or any(pp.result.iloc[:3,0] == 'cp_0'):\n    continue\n  else:\n    print(str(i-1))\n"

# Analiza porównawcza z porzednim przypadkiem
Porównuję do poprzedniej obserwacji.

W przypadu  tego 59 letniego męszczyzny najważniejszymi zmiennymi są:
- obniżenie ST na elektrogramie
- ciśnienie spoczynkowe

Przewidywania te są nieefektywne ponieważ w rzeczywistości pacjent jest chory. 
Ogolnie tak duży spadek ST wskazuje na problemy z sercem przeciwnie do tego co wskazuje mój model, tak samo duże ciśnienie powinno przemawiać za występowaniem choroby. Duży spadek prawdopodobieństwa wystąpienia choroby jest spowodowany istniniem nieodwracalnej zmiany serca. 
Ten przykład wskazuje, że ten model jest źle zaprojektowany. 


## 3

In [ ]:
pp = explainer.predict_parts(x_test.iloc[15])
pp.plot()

# Wpływ zmiennych na wynik predykcji
W tym przypadku próbowałem wybrać dwie osoby dla których ta sama zminna wpływa przeciwnie, ale po przeszukaniu praktycznie wszystkich obserwacji nie udało mi się takiej znaleźć. Jest to dość nieintuicyjne dlatego postanowiłem sprawdzić czy poszczególne zminne. Kodem poniżej sprawdzam, czy istnieje obserwacja która posiada thal_2 = 0 i ma wpływ dodatni na *prediction*. Nie istnieje, w taki sam spobób sprawdziłem wszystkie inne zmienne kategoryczne.

In [89]:
for i in range(1,x_train.shape[0]):
  pp = explainer.predict_parts(x_train.iloc[i-1], B = 1)
  v = pp.result.loc[pp.result.variable_name == 'thal_1',['variable_value']]
  s = pp.result.loc[pp.result.variable_name == 'thal_1',['sign']]
  if v.iloc[0,0] == '0.0' and s.iloc[0,0] == 1:
    print(i)
  if v.iloc[0,0] == '1.0'  and s.iloc[0,0] == -1:
    print(i)
